In [2]:
from functools import partial
from typing import List, Tuple, Union

import numpy as np
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display

In [3]:
X = +1
O = -1
EMPTY_SPACE = 0

INTERFACE_MAPPING = {
    X: 'X',
    O: 'O'
}

In [4]:
class Interface:
    def __init__(self):
        self.buttons = [
            [
                widgets.Button(
                    description='',
                    layout=Layout(width='100px', height='100px')
                )
                for _ in range(3)
            ]
            for _ in range(3)
        ]

        self.board_widget = VBox([HBox(row) for row in self.buttons])

        self.__player = X
        self.link_positions()

    def on_select_position(self, pos: tuple, button: widgets.Button):
        """Callback on user click on a position of the board

        :param pos: row, column of button on grid
        :param button: clicked button
        """
        if not button.description:
            button.description = INTERFACE_MAPPING.get(self.__player, '')
            self.__player *= -1  # invert player

    def link_positions(self):
        """Link clicks on buttons"""
        for i, row in enumerate(self.buttons):
            for j, button in enumerate(row):
                button.on_click(partial(self.on_select_position, (i, j)))

    def disable_buttons(self):
        """Disable buttons to avoid clicks after end game"""
        for row in self.buttons:
            for button in row:
                button.disabled = True


    def start(self):
        """Display board interface"""
        display(self.board_widget)

In [5]:
interface = Interface()
interface.start()

In [6]:
board = np.zeros((3, 3), dtype=int)
board[:, 1] = O
np.diag(board)
board

array([[ 0, -1,  0],
       [ 0, -1,  0],
       [ 0, -1,  0]])

In [16]:
def get_game_status(board: np.ndarray) -> Tuple[bool, Union[int, None]]:

    """Check if game ended and who is the winner

    :param board:
    :return: if its a game over, who is the winner
    """
     # Check rows
    for row in board:
        if np.sum(row) == 3:  # Player 1 wins
            return True, 1
        elif np.sum(row) == -3:  # Player -1 wins
            return True, -1

    # Check columns
    for col in board.T:  # Transpose to iterate over columns as rows
        if np.sum(col) == 3:  # Player 1 wins
            return True, 1
        elif np.sum(col) == -3:  # Player -1 wins
            return True, -1

    # Check main diagonal
    main_diag = np.sum(np.diag(board))
    if main_diag == 3:
        return True, 1
    elif main_diag == -3:
        return True, -1

    # Check anti-diagonal
    anti_diag = np.sum(np.diag(np.fliplr(board)))
    if anti_diag == 3:
        return True, 1
    elif anti_diag == -3:
        return True, -1

    # Check if there are still empty spaces (game not over)
    if np.any(board == 0):
        return False, None

    # Game is a tie
    return True, None

In [17]:
def get_possible_moves(board: np.ndarray, player: int = X) -> List[np.ndarray]:
    """Get next possible moves by some player
    """
    possible_moves = []
    for row in range(board.shape[0]):
        for col in range(board.shape[1]):
            if board[row, col] == 0:  # Assumindo que 0 indica uma posição vazia
                new_board = board.copy()
                new_board[row, col] = player
                possible_moves.append(new_board)
    return possible_moves
    pass

In [24]:
AI_PLAYER = X
HUMAN_PLAYER = O
MAX_N_MOVES = 9


def get_score(winner: int, n_moves: int) -> int:
    """Get how well was the game for the AI:
        - win faster is better than win slower
        - lose slower is better than win faster
        - draw is a intermediary result
    """
    pass


In [36]:
def mini_max(board, player=AI_PLAYER, n_moves=0):
    is_over, winner = get_game_status(board)  # Verifique o estado do jogo
    if is_over:  # Se o jogo acabou, retorne o score
        score = get_score(winner, n_moves)  # Obtenha a pontuação com base no vencedor
        return score, board

    possible_moves = get_possible_moves(board, player)  # Obtenha os movimentos possíveis

    best_score = float('-inf') if player == AI_PLAYER else float('inf')  # Inicialize a melhor pontuação
    best_move = None

    for move in possible_moves:  # Para cada movimento possível
        score, _ = mini_max(move, -player, n_moves + 1)  # Chame minimax recursivamente

        if player == AI_PLAYER:  # Se for a vez da IA
            if score > best_score:  # Se a pontuação for melhor que a melhor pontuação atual
                best_score = score  # Atualize a melhor pontuação
                best_move = move  # Atualize o melhor movimento
        else:  # Se for a vez do humano
            if score < best_score:  # Se a pontuação for melhor que a melhor pontuação atual
                best_score = score  # Atualize a melhor pontuação
                best_move = move  # Atualize o melhor movimento

    return best_score, best_move  # Retorne a melhor pontuação e o melhor movimento
    pass

In [37]:
board = np.zeros(shape=(3, 3), dtype=np.int8)
_, new_board = mini_max(board, AI_PLAYER)
new_board

array([[1, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int8)

In [33]:
new_board = new_board.copy()
new_board[1, 0] = -1
_, new_board = mini_max(new_board, AI_PLAYER)
new_board

array([[ 1,  1,  0],
       [-1,  0,  0],
       [ 0,  0,  0]], dtype=int8)

In [34]:
END_GAME_MESSAGES = {
    AI_PLAYER: 'AI won!',
    HUMAN_PLAYER: 'You won!',
    EMPTY_SPACE: "It is a draw"
}


class TicTacToeAI(Interface):
    def __init__(self, ai_starts=False):
        super().__init__()
        self.board = np.zeros(shape=(3, 3), dtype=np.int8)
        if ai_starts:
            self.board = mini_max(self.board, AI_PLAYER)[1].copy()
            self.update()


    def on_select_position(self, pos: tuple, button: widgets.Button):
        """Callback on user click on a position of the board
        It calls minmax algorithm after each user move

        :param pos: row, column of button on grid
        :param button: clicked button"""
        if not button.description:
            self.board[pos] = HUMAN_PLAYER
            self.board = mini_max(self.board, AI_PLAYER)[1].copy()

            is_over, winner = get_game_status(self.board)
            if is_over:
                self.disable_buttons()
                print(END_GAME_MESSAGES.get(winner))

            self.update()

    def update(self):
        """Update interface from virtual board"""
        for i, row in enumerate(self.board):
            for j, item in enumerate(row):
                self.buttons[i][j].description = INTERFACE_MAPPING.get(item, '')


In [40]:
game = TicTacToeAI(ai_starts=False)
game.start()

None
